In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
import json
from nltk.stem import PorterStemmer
from sklearn import linear_model
import nltk
from nltk.corpus import stopwords
import tensorflow as tf


2023-11-29 21:24:54.485110: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)

In [3]:
users_reviews = [l for l in parse("review-Hawaii_10.json.gz")]
businesses = [l for l in parse("meta-Hawaii.json.gz")]

In [4]:
len(users_reviews)

1504347

In [5]:
users_reviews[0]

{'user_id': '113965417079576625433',
 'name': 'manuel grimaldo',
 'time': 1591839903487,
 'rating': 5,
 'text': 'Great new upgrade',
 'pics': None,
 'resp': None,
 'gmap_id': '0x7c00159b5b1b1d25:0x8d2d85d4a758290e'}

In [6]:
userIDs = {}
itemIDs = {}
interactions = []

for d in users_reviews:
    u = d['user_id']
    i = d['gmap_id']
    r = d['rating']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)
    interactions.append((u,i,r))
        

In [7]:
random.shuffle(interactions)
len(interactions)

1504347

In [8]:
nTrain = int(len(interactions) * 0.9)
interactionsTrain = interactions[:nTrain]
interactionsTest = interactions[nTrain:]

In [9]:
itemsPerUser = defaultdict(list)
usersPerItem = defaultdict(list)
for u,i,r in interactionsTrain:
    itemsPerUser[u].append(i)
    usersPerItem[i].append(u)

In [10]:
mu = sum([r for _,_,r in interactionsTrain]) / len(interactionsTrain)

In [11]:
optimizer = tf.keras.optimizers.Adam(0.1)

In [12]:
class LatentFactorModel(tf.keras.Model):
    def __init__(self, mu, K, lamb):
        super(LatentFactorModel, self).__init__()
        # Initialize to average
        self.alpha = tf.Variable(mu)
        # Initialize to small random values
        self.betaU = tf.Variable(tf.random.normal([len(userIDs)],stddev=0.001))
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(itemIDs),K],stddev=0.001))
        self.lamb = lamb

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        p = self.alpha + self.betaU[u] + self.betaI[i] +\
            tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.reduce_sum(self.betaU**2) +\
                            tf.reduce_sum(self.betaI**2) +\
                            tf.reduce_sum(self.gammaU**2) +\
                            tf.reduce_sum(self.gammaI**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        pred = self.alpha + beta_u + beta_i +\
               tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        return tf.nn.l2_loss(pred - r) / len(sampleR)

In [23]:
modelLFM = LatentFactorModel(mu, 8, 0.001)

In [24]:
# Training step (for the batch-based model from Chapter 5)
def trainingStep(model, interactions):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleR = [], [], []
        for _ in range(Nsamples):
            u,i,r = random.choice(interactions)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleR.append(r)

        loss = model(sampleU,sampleI,sampleR)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

In [25]:
# Run 100 iterations (really 100 batches) of gradient descent
for i in range(100):
    obj = trainingStep(modelLFM, interactionsTrain)
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

iteration 10, objective = 0.61733377
iteration 20, objective = 0.6224872
iteration 30, objective = 0.54347044
iteration 40, objective = 0.47025475
iteration 50, objective = 0.45126355
iteration 60, objective = 0.44041047
iteration 70, objective = 0.44370794
iteration 80, objective = 0.44013566
iteration 90, objective = 0.44147405
iteration 100, objective = 0.44104448


In [26]:
def MSE(predictions, labels):
    differences = [(x - y) ** 2 for x, y in zip(predictions, labels)]
    return sum(differences) / len(differences)

In [27]:
test_y = [r for u,i,r in interactionsTest]

In [28]:
predictions = [modelLFM.predict(userIDs[u],itemIDs[i]).numpy() for u,i,r in interactionsTest]

In [31]:
test_y[0]
predictions[0]

4.4092007

In [29]:
mse = MSE(predictions, test_y)
mse

0.831411004044466